In [2]:
# https://docs.bittensor.com/subnets/register-validate-mine#check-the-permit-status
import bittensor as bt
import asyncio

# NET_UID = 61
NET_UID = 170
NETWORK = "test"
WALLET = "validator"
wallet = bt.wallet(name=WALLET)
subtensor = bt.subtensor(network=NETWORK)
subnet = subtensor.metagraph(netuid=NET_UID)

hotkey = wallet.hotkey.ss58_address
my_uid = subnet.hotkeys.index(wallet.hotkey.ss58_address)
sub = bt.subtensor(NETWORK)
mg = sub.metagraph(NET_UID)
if hotkey not in mg.hotkeys:
    print(f"Hotkey {hotkey} deregistered")
else:
    print(f"Hotkey {hotkey} is registered. UID: {my_uid}")
active_uids = subnet.uids.tolist()
print(f"Active UIDs (total: {len(active_uids)}): {active_uids}")
print(f"My hotkey is active: {my_uid in active_uids}")
print(f"Is val permit: {subnet.validator_permit[my_uid]}")
print(f"1000+ TAO:")

Hotkey 5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr is registered. UID: 0


In [11]:
# Sample validators.
# TODO: Stress test: how many queries
from prompting.protocol import StreamPromptingSynapse
from prompting.forward import handle_response
from transformers import AutoTokenizer

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = ["user"]
messages = ["Organic prompt query. Question: Capital of Zimbabwe?"]
timeout = 40
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)

streams_responses = await dendrite(
    axons=axons,
    synapse=StreamPromptingSynapse(roles=roles, messages=messages),
    timeout=timeout,
    deserialize=False,
    streaming=True,
)

stream_results_dict = dict(zip(uids, streams_responses))
handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
stream_results = await handle_stream_responses_task

bt.logging.info(f"Response:\n {stream_results[0].synapse.completion}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Response:
 [SynapseStreamResult(exception=None, uid=0, accumulated_chunks=['The capital of Zimbabwe is Harare.'], accumulated_chunks_timings=[6.252922296524048], tokens_per_chunk=[8], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['Organic prompt query. Question: Capital of Zimbabwe?'], completion='The capital of Zimbabwe is Harare.'))] - 


In [21]:
# TODO: Stress test: how many queries
from prompting.protocol import StreamPromptingSynapse
from prompting.forward import handle_response
from transformers import AutoTokenizer

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = []
messages = []
timeout = 40
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
while True:
    message = input()
    if message == "exit":
        break
    roles.append("user")
    messages.append(message)
    streams_responses = await dendrite(
        axons=axons,
        synapse=StreamPromptingSynapse(roles=roles, messages=messages),
        timeout=timeout,
        deserialize=False,
        streaming=True,
    )
    stream_results_dict = dict(zip(uids, streams_responses))
    handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
    stream_results = await handle_stream_responses_task
    response = stream_results[0].synapse.completion

    print(f"User: {message}")
    print(f"Assistant {response}")

    messages.append(response)
    roles.append("assistant")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


User: Hey my name is John, and I live in Zimbabwe
Assistant Hello, John! It's nice to meet you. How can I assist you today?


In [13]:
stream_results[0].synapse.completion

'The capital of Zimbabwe is Harare.'

In [53]:
stream_results

[SynapseStreamResult(exception=UnboundLocalError("local variable 'chunk' referenced before assignment"), uid=166, accumulated_chunks=[], accumulated_chunks_timings=[], tokens_per_chunk=[], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['failure'], completion=''))]

In [44]:
response_event

DendriteResponseEvent(uids=[166], completions=[''], timings=[0], status_messages=['Service at 194.68.245.74:8091/StreamPromptingSynapse unavailable.'], status_codes=[503])

In [35]:
streams_responses

[<async_generator object DendriteMixin.call_stream at 0x7f2dc3d30fc0>]